In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime

In [5]:
#スクレイピングしたデータを入れる表を作成
list_df = pd.DataFrame(columns=['曲名','発売日', '表示回数', '歌詞'])

# アーティストごとにページ数が違うので、rangeを変更する
for page in range(1,2):
    base_url = 'https://www.uta-net.com'
    #歌詞一覧ページ
    url = base_url + '/artist/31108/0/' + str(page) + '/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    # links = soup.find_all('td', class_='side td1')
    links = soup.find_all('td', class_='sp-w-100 pt-0 pt-lg-2')


for link in links:
    a = base_url + (link.a.get('href'))
    # 歌詞詳細ページ
    response = requests.get(a)

    soup = BeautifulSoup(response.text, 'lxml')
    # 曲名を取得
    song_name = soup.find('h2').text
    # 発売日、表示回数などを取得
    detail = soup.find('p', class_="detail").text
    # 発売日を取得
    match = re.search(r'\d{4}/\d{2}/\d{2}', detail)
    release_date = datetime.strptime(match.group(), '%Y/%m/%d').date()
    # 表示回数を取得
    p = r'この曲の表示回数：(.*)回'
    impressions = re.search(p, detail).group(1)
    # 歌詞を取得
    song_lyrics = soup.find('div', itemprop='lyrics')
    song_lyric = song_lyrics.text
    # 改行を削除
    song_lyric = song_lyric.replace('\n','')
    # 以下の文言を削除
    song_lyric = song_lyric.replace('この歌詞をマイ歌ネットに登録 >このアーティストをマイ歌ネットに登録 >','')

    #サーバーに負荷を与えないため1秒待機
    time.sleep(1)

    #取得した歌詞を表に追加
    tmp_se = pd.DataFrame([[song_name], [release_date], [impressions], [song_lyric]], index=list_df.columns).T
    list_df = list_df.append(tmp_se)

<ipython-input-5-1978cc0ad617>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_df = list_df.append(tmp_se)
<ipython-input-5-1978cc0ad617>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  list_df = list_df.append(tmp_se)


KeyboardInterrupt: 

In [4]:
#csv保存
list_df.to_csv('./output/list.csv', mode = 'w', encoding='utf-8')

#### 参考
- [ミスチルの歌詞をスクレイピングで取得してみた【Beautiful Soup】](https://zatsugaku-engineer.com/python/scraping/)